In [1]:
from collections import Counter
import pandas as pd
import sklearn as sk
import math 
import pprint
import requests
from bs4 import BeautifulSoup
import pandas as pd
import scipy.spatial
from sentence_transformers import SentenceTransformer, util

df = pd.read_csv(r'data/PS1.csv')

df[['name', 'industrydesc']]

def cosine(vector1, vector2):
    # calculate nominator as a dot product
    numerator = sum(map(lambda pair:pair[0]*pair[1],zip(vector1, vector2)))
    
    # calculate the denominator, use Euclidean norm
    sum1 = sum([vector1[i] ** 2 for i in list(vector1.keys())])
    sum2 = sum([vector2[i] ** 2 for i in list(vector2.keys())])   
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    
    if not denominator:
         return 0.0
    else:
         return float(numerator)/denominator   

def cvect(sent):
    wvec = dict.fromkeys(all, 0) 
    for word in sent:
        wvec[word]+=1
    return wvec

for (index, row), ii in zip(df.iterrows(), range(10)):
    company_sentence1 = df.iloc[ii+1]["industrydesc"]
    company_sentence1 = company_sentence1.split(" ")
    for index, row in df.iterrows():
        company_sentence2 = row["industrydesc"]
        company_sentence2 = company_sentence2.split(" ")
        all = set(company_sentence2).union(set(company_sentence1))
        dict1 = cvect(company_sentence1)
        dict2 = cvect(company_sentence2)
        dfc = pd.DataFrame([dict1, dict2])
        corr = cosine(dfc.iloc[0], dfc.iloc[1])
        print("Similarity: ", corr)
        print(len(all))
        print(f'{all} \n \n')

Similarity:  0.0
4
{'vedrørende', 'informationsteknologi', 'Livsforsikring', 'Konsulentbistand'} 
 

Similarity:  1.0
1
{'Livsforsikring'} 
 

Similarity:  0.0
10
{'navigation', 'og', 'Livsforsikring', 'udstyr', 'måling,', 'til', 'Fremstilling', 'afprøvning,', 'af', 'kontrol'} 
 

Similarity:  0.0
7
{'dele', 'Fremstilling', 'og', 'hertil', 'Livsforsikring', 'af', 'høreapparater'} 
 

Similarity:  0.0
3
{'holdingselskaber', 'Finansielle', 'Livsforsikring'} 
 

Similarity:  0.0
2
{'Computerprogrammering', 'Livsforsikring'} 
 

Similarity:  0.0
4
{'vedrørende', 'informationsteknologi', 'Livsforsikring', 'Konsulentbistand'} 
 

Similarity:  0.0
5
{'Fremstilling', 'Livsforsikring', 'præparater', 'af', 'farmaceutiske'} 
 

Similarity:  0.0
2
{'Computerprogrammering', 'Livsforsikring'} 
 

Similarity:  0.0
4
{'vedrørende', 'informationsteknologi', 'Livsforsikring', 'Konsulentbistand'} 
 

Similarity:  0.0
4
{'af', 'Udgivelse', 'Livsforsikring', 'bøger'} 
 

Similarity:  0.0
3
{'telekommunikat

In [2]:
urls = ['https://www.kmd.dk/','https://pfa.dk/privat/', 'https://dynatest.com/', 
                    'https://www.oticon.dk/hearing-aid-users','https://www.topdanmark.dk/',
                    'https://northtech.dk/','https://www.netcompany.com/da','https://www.novonordisk.dk/',
                    'https://www.formpipe.com/da/','https://www.alpha-solutions.com','https://dafolo.dk/', 
                    'https://nuuday.dk/']

df['website'] = urls

# save the dataframe to a csv file
df.to_csv('data/clean_data.csv', index=False) # set index=False to exclude index column in the saved file

for index, row in df.iterrows():
    
    response = requests.get(row['website'])
    web = response.text
    soup = BeautifulSoup(web,'html.parser')
    corpus = soup.find_all('p')
    texts = [p.text for p in corpus]
    print(texts)

    embedder = SentenceTransformer('all-MiniLM-L6-v2')
    corpus_embeddings = embedder.encode(texts)
    corpus_embeddings


    queries = ["Søger i praktikanter?", "programmering", "kan jeg låne bøger?",]
    query_embedding = embedder.encode(queries)
    query_embedding

    closest_n = 3
    for query, query_embedding in zip(queries, query_embedding):
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])

        print("\n======================\n")
        print("Query:", query)
        print("\nTop 3 most similar sentences in corpus:\n")

        for idx, distance in results[0:closest_n]:
            print(texts[idx].strip(), "(Score: %.4f)" % (1-distance))

    similarity = util.dot_score(query_embedding, corpus_embeddings)
    similarity

['\r\n                    Vi hjælper jeres organisation med at løse morgendagens udfordringer gennem professionelle og tilpassede it-løsninger. Vores løsninger dækker blandt andet over særligt udviklede brancheløsninger samt software og strategisk rådgivning, der løfter jeres organisations kernefunktioner til nye højder.\r\n                ', 'Energi & forsyning', 'Vi har et langvarigt og dybdegående kendskab til energi- og forsyningssektoren. Vi har et bredt kendskab til kompleksitet, data og processer samt lovgivning og regulativer der bidrager til omskifteligheden inden for forsyningsbranchen.', 'Kommune', 'Vi har gennem årtier arbejdet sammen med kommunerne med fokus på at sætte standarden for et digitaliseret samfund med åbne og fremtidssikrede løsninger. ', 'Stat', 'Hvis vi ønsker at fastholde Danmarks internationale førerposition inden for offentlig digitalisering, kræver det, at vi som samfund bliver bedre til at udnytte vores data. Også i den statslige sektor. I den proces stå



Query: Søger i praktikanter?

Top 3 most similar sentences in corpus:

Jordskælv i Tyrkiet og Syrien (Score: 0.6238)
I PFA følger der en lang række fordele med din opsparing - uanset om du indbetaler til den eller ej (Score: 0.6198)
I PFA følger der en lang række fordele med din opsparing - uanset om du indbetaler til den eller ej (Score: 0.6198)


Query: programmering

Top 3 most similar sentences in corpus:

Nyttige links (Score: 0.1887)
Jordskælv i Tyrkiet og Syrien (Score: 0.1645)
Med investeringsguiden på Mit PFA kan du vælge den investeringsprofil, der er rigtig for dig (Score: 0.1645)


Query: kan jeg låne bøger?

Top 3 most similar sentences in corpus:

Jordskælv i Tyrkiet og Syrien (Score: 0.6197)
I PFA følger der en lang række fordele med din opsparing - uanset om du indbetaler til den eller ej (Score: 0.6010)
I PFA følger der en lang række fordele med din opsparing - uanset om du indbetaler til den eller ej (Score: 0.6010)
['\n                        Let us tell you why!  

['Er du blevet ramt af stormen? Anmeld din skade i\xa0Mit Topdanmark', 'Vælg en af landets bedste bilforsikringer - også til el- og hybridbiler - og bliv elitebilist på 3 år, uanset din alder.', 'Anmeld alt fra vejrskade til indbrud', 'Du kan finde svar på det meste', 'Køb eller få tilbud - nemt og hurtigt', 'Har du samlet dine forsikringer hos os? Så er det nemt at få adgang til Sundhedshjælp. Du kan enten hente appen eller få adgang på din computer.\xa0', 'Hvis din elbil eller plug-in-hybidbil er forsikret hos Topdanmark, kan du spare 500 kr. på din ladeløsning og installation fra Norlys.', 'Vi vil gerne give dig den bedste service og oplevelse på topdanmark.dk. Derfor har vi oprettet et brugerpanel på Facebook, hvor du som medlem løbende vil blive inviteret til en række brugerundersøgelser.\xa0', 'Deltager du i en brugerundersøgelse, sender vi dig et gavekort som tak for hjælpen. \xa0\n\xa0', '\nAnsvarsbegrænsning\nPrivatlivspolitik\nCookies\n']


Query: Søger i praktikanter?

Top 3

ConnectionError: HTTPSConnectionPool(host='www.formpipe.com', port=443): Max retries exceeded with url: /da/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000226196BD7F0>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))